<a href="https://colab.research.google.com/github/danielhou13/cogs402longformer/blob/main/src/Attn_attr_cosine_sim_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook explores the relation between the model's attributions and attentions for a given example. Historically, we found that attentions are not a feasible method of explanation whereas attributions are, but attributions are also not part of a model's traditional outputs. Therefore it may be interesting to see if we can find anything with attentions by comparing them to a feasible and plausible method of explanation.

In [117]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import dependencies

In [118]:
pip install transformers --quiet

In [119]:
pip install captum --quiet

In [120]:
pip install datasets --quiet

In [121]:
pip install rbo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [122]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [123]:
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

import torch
import pandas as pd

In [124]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Import model

In [125]:
from transformers import LongformerForSequenceClassification, LongformerTokenizer, LongformerConfig
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model
model_path = 'danielhou13/longformer-finetuned_papers_v2'
#model_path = 'danielhou13/longformer-finetuned-new-cogs402'

# load model
model = LongformerForSequenceClassification.from_pretrained(model_path, num_labels = 2)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

Create functions that give us the input ids and the position ids for the text we want to examine

In [126]:
def predict(inputs, position_ids=None, attention_mask=None):
    output = model(inputs,
                   position_ids=position_ids,
                   attention_mask=attention_mask)
    return output.logits

In [127]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [128]:
max_length = 2046
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, truncation = True, add_special_tokens=False, max_length = max_length)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)

    #taken from the longformer implementation
    mask = input_ids.ne(ref_token_id).int()
    incremental_indices = torch.cumsum(mask, dim=1).type_as(mask) * mask
    position_ids = incremental_indices.long().squeeze() + ref_token_id

    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

Import dataset and take a few examples from it for testing purposes

Here we import the papers dataset

In [129]:
from datasets import load_dataset
import numpy as np
cogs402_ds = load_dataset("danielhou13/cogs402dataset")["test"]

Using custom data configuration danielhou13--cogs402dataset-144b958ac1a53abb
Reusing dataset parquet (/root/.cache/huggingface/datasets/danielhou13___parquet/danielhou13--cogs402dataset-144b958ac1a53abb/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)


  0%|          | 0/2 [00:00<?, ?it/s]

Here we import the news dataset

In [ ]:
# cogs402_ds2 = load_dataset('hyperpartisan_news_detection', 'bypublisher')['validation']
# val_size = 5000
# val_indices = np.random.randint(0, len(cogs402_ds2), val_size)
# val_ds = cogs402_ds2.select(val_indices)
# labels2 = map(int, val_ds['hyperpartisan'])
# labels2 = list(labels2)
# val_ds = val_ds.add_column("labels", labels2)

In [130]:
#set 1 if we are dealing with a positive class, and 0 if dealing with negative class
def custom_forward(inputs, position_ids=None, attention_mask=None):
    preds = predict(inputs,
                   position_ids=position_ids,
                   attention_mask=attention_mask
                   )
    return torch.softmax(preds, dim = 1)

Perform Layer Integrated Gradients using the longformer's embeddings

In [131]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    return attributions

In [132]:
lig = LayerIntegratedGradients(custom_forward, model.longformer.embeddings)

This function will let us get the example and the baseline inputs in order to perform integrated gradients, and add the attributions to our visualization tool. Additionally, we will add the attributions and tokens for each example into an array so we can use them when we want to further example the attributions scores for each example

In [ ]:
all_attributions = {}
# all_tokens = {}

In [133]:
all_attributions = torch.load('/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/example_attrib_dict_all.pt')

In [ ]:
# from tqdm import tqdm

# for i in tqdm(range(len(cogs402_ds))):
#   if str(i) not in all_attributions:
#     #get input ids, position ids and attention mask for integrated gradients
#     text = cogs402_ds['text'][i]
#     label = cogs402_ds['labels'][i]

#     input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
#     position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
#     attention_mask = construct_attention_mask(input_ids)

#     attributions = lig.attribute(inputs=input_ids,
#                                       baselines=ref_input_ids,
#                                       additional_forward_args=(position_ids, attention_mask),
#                                       target=1,
#                                       n_steps=50,
#                                       internal_batch_size = 2)

#     attributions_sum = summarize_attributions(attributions)

#     all_attributions[str(i)] = attributions_sum.detach().cpu().numpy()

#     torch.save(all_attributions, '/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/example_attrib_dict_all.pt')

We then get the attentions and global attentions so we can compare with the attributions

Since the longformer has a unique attention matrix shape, we convert it into the required sequence length x sequence length matrix

In [134]:
def create_head_matrix(output_attentions, global_attentions):
    new_attention_matrix = torch.zeros((output_attentions.shape[0], 
                                      output_attentions.shape[0]))
    for i in range(output_attentions.shape[0]):
        test_non_zeroes = torch.nonzero(output_attentions[i]).squeeze()
        test2 = output_attentions[i][test_non_zeroes[1:]]
        new_attention_matrix_indices = test_non_zeroes[1:]-257 + i
        new_attention_matrix[i][new_attention_matrix_indices] = test2
        new_attention_matrix[i][0] = output_attentions[i][0]
        new_attention_matrix[0] = global_attentions.squeeze()[:output_attentions.shape[0]]
    return new_attention_matrix


def attentions_all_heads(output_attentions, global_attentions):
    new_matrix = []
    for i in range(output_attentions.shape[0]):
        matrix = create_head_matrix(output_attentions[i], global_attentions[i])
        new_matrix.append(matrix)
    return torch.stack(new_matrix)

def all_layers(output_attentions, global_attentions):
    new_matrix = []
    for i in range(output_attentions.shape[0]):
        matrix = attentions_all_heads(output_attentions[i], global_attentions[i])
        new_matrix.append(matrix)
    return torch.stack(new_matrix)

We scale the attention matrix by head importance

In [135]:
def scale_by_importance(attention_matrix, head_importance):
  new_matrix = np.zeros_like(attention_matrix)
  for i in range(attention_matrix.shape[0]):
    head_importance_layer = head_importance[i]
    new_matrix[i] = attention_matrix[i] * np.expand_dims(head_importance_layer, axis=(1))
  return new_matrix

In [136]:
head_importance = torch.load("/content/drive/MyDrive/cogs402longformer/t3-visapplication/resources/papers/pretrained/head_importance.pt")
# head_importance = torch.load("/content/drive/MyDrive/cogs402longformer/t3-visapplication/resources/news/head_importance.pt")

In [ ]:
# print(all_attentions_final)

The following block of code creates a new dictionary of attention matrices. Each key corresponds to their respective example in the dataset (range 0 - dataset length) and stores a layer x head x seq_len matrix of attentions for each key

In [ ]:
all_attentions = {}

The following loads a saved dictionary of attention matrices without having applied head importance to the matrices.

In [137]:
all_attentions = torch.load('/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/full_attention_matrices/papers_atten_summed_dict.pt')

In [ ]:
from tqdm import tqdm

for i in tqdm(range(len(cogs402_ds))):
  if str(i) not in all_attentions:
    #get input ids, position ids and attention mask for integrated gradients
    text = cogs402_ds['text'][i]
    label = cogs402_ds['labels'][i]

    input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
    position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
    attention_mask = construct_attention_mask(input_ids)

    output = model(input_ids.cuda(), attention_mask=attention_mask.cuda(), labels=torch.tensor(label).cuda(), output_attentions = True)

    batch_attn = output[-2]
    output_attentions = torch.stack(batch_attn).cpu().squeeze()
    global_attention = output[-1]
    output_global_attentions = torch.stack(global_attention).cpu().squeeze()

    converted_mat = all_layers(output_attentions, output_global_attentions).detach().cpu().numpy()
    
    attention_matrix_summed = converted_mat.sum(axis=2)
    all_attentions[str(i)] = attention_matrix_summed

    if i%10 == 9:
      torch.save(all_attentions, '/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/full_attention_matrices/papers_atten_summed_dict.pt')

100%|██████████| 1070/1070 [11:55:01<00:00, 40.09s/it]


In [ ]:
all_attentions = torch.load('/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/full_attention_matrices/papers_atten_summed_dict.pt')

The following are two dictionaries of attention weights for each token (how much each token is attended to), weighted by head importance, for layer 12 and over all layers. The number of keys in the dictionary is 1070 (number of items in the validation set) and each key contains an array of shape (seq_len,).

In [138]:
all_attentions_final = torch.load('/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/full_attention_matrices/example_atten_dict_12.pt')
all_attentions_all = torch.load('/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/full_attention_matrices/example_atten_dict_all.pt')

In [ ]:
all_attentions_final = {}
all_attentions_all = {}
for i in tqdm(range(len(cogs402_ds))):
  if str(i) not in all_attentions_final and str(i) not in all_attentions_all:

    att_mat = all_attentions[str(i)]

    # uncomment the line if you wish to scale by head importance
    att_mat = scale_by_importance(att_mat, head_importance)

    # Sum the attentions for the last layer and over all layers
    attention_final_layer = att_mat[11].sum(axis=0)
    all_attentions_final[str(i)] = attention_final_layer

    attention_all_layer = att_mat.sum(axis=1)
    attention_all_layer = attention_all_layer.sum(axis=0)
    all_attentions_all[str(i)] = attention_all_layer
    
    torch.save(all_attentions_final, '/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/full_attention_matrices/example_atten_dict_12.pt')
    torch.save(all_attentions_all, '/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/full_attention_matrices/example_atten_dict_all.pt')

100%|██████████| 1070/1070 [02:59<00:00,  5.97it/s]


In [139]:
def normalize(data):
    return (data - np.min(data)) / (np. max(data) - np.min(data))

The following block of code iterates through the entire dataset, grabs the appropriate attributions and attentions from their respective dictionaries, and computes the cosine similarities, kendall tau coefficients, and the RBO.

In [156]:
from numpy.linalg import norm
import scipy.stats as stats
import rbo

In [178]:
def get_sim_dataframe(cogs402_ds, all_attentions, all_attributions):

  dataframe = []

  for i in tqdm(range(len(cogs402_ds))):
    exam_attrib = all_attributions[str(i)]
    attention_final_layer = all_attentions[str(i)]
    
    exam_attrib = exam_attrib[:len(all_attentions[str(i)])]

    #raw attributions
    cosine_raw = np.dot(exam_attrib, attention_final_layer) / (norm(exam_attrib)*norm(attention_final_layer))

    attention_final_layer2 = normalize(attention_final_layer)

    exam_attrib2 = np.abs(exam_attrib)
    exam_attrib2 = normalize(exam_attrib2)
    
    #normalized attributions
    cosine = np.dot(exam_attrib2, attention_final_layer2) / (norm(exam_attrib2)*norm(attention_final_layer2))

    exam_attrib3 = np.abs(exam_attrib)
    exam_attrib3 = normalize(exam_attrib3)
    median_exam = np.percentile(exam_attrib3, 50)
    exam_attrib3[exam_attrib3 < median_exam] = 0

    attention_final_layer3 = np.copy(attention_final_layer)
    attention_final_layer3 = normalize(attention_final_layer3)
    median_12 = np.percentile(attention_final_layer3, 50)
    attention_final_layer3[attention_final_layer3 < median_12] = 0


    #sim for attributions and attentions above the median
    cosine_med = np.dot(exam_attrib3, attention_final_layer3) / (norm(exam_attrib3)*norm(attention_final_layer3))

    exam_attrib4 = np.abs(exam_attrib)
    exam_attrib4 = normalize(exam_attrib4)
    mean_exam = np.mean(exam_attrib4)
    exam_attrib4[exam_attrib4 < mean_exam] = 0

    attention_final_layer4 = np.copy(attention_final_layer)
    attention_final_layer4 = normalize(attention_final_layer4)
    mean_12 = np.mean(attention_final_layer4)
    attention_final_layer4[attention_final_layer4 < mean_12] = 0

    #sim for attributions and attentions above the mean
    cosine_mean = np.dot(exam_attrib4, attention_final_layer4) / (norm(exam_attrib4)*norm(attention_final_layer4))

    exam_attrib_rank = np.abs(exam_attrib)
    order_attrib = exam_attrib_rank.argsort()
    ranks_attrib = order_attrib.argsort()

    attention_final_layer_rank = np.copy(attention_final_layer)
    order = attention_final_layer_rank.argsort()
    ranks = order.argsort()

    #sim using the ranks of the tokens
    cosine_rank = np.dot(ranks_attrib, ranks) / (norm(ranks_attrib)*norm(ranks))

    tau, p_value = stats.kendalltau(ranks_attrib, ranks)

    rbo_sim = rbo.RankingSimilarity(order_attrib, order).rbo()

    d = {'example': i, 'similarity normalized': cosine, 'similarity raw': cosine_raw, 'sim_norm w/ median threshold':cosine_med, 'sim_norm w/ mean threshold':cosine_mean, "sim w/ ranks":cosine_rank,
        "kendalltau": tau, "rbo":rbo_sim}
    dataframe.append(d)

  return pd.DataFrame(dataframe)

The attributions and the attentions have different ranges. The attributions could range from -1 to 1 whereas the attentions range from 0 to 1. However, negative attributions would not necessarily mean that they have the lowest attention, rather they might have really high attention as they are more likely to help the model predict the negative class, and might be something the attentions picked up as a feature.

In [179]:
df = get_sim_dataframe(cogs402_ds, all_attentions_final, all_attributions)

100%|██████████| 1070/1070 [00:07<00:00, 142.41it/s]


In [187]:
df2 = get_sim_dataframe(cogs402_ds, all_attentions_all, all_attributions)

100%|██████████| 1070/1070 [00:07<00:00, 142.64it/s]


In [188]:
df

,example,similarity normalized,similarity raw,sim_norm w/ median threshold,sim_norm w/ mean threshold,sim w/ ranks,kendalltau,rbo
0,0,0.086231,0.056545,0.085778,0.080140,0.846132,0.263652,0.588661
1,1,0.303640,-0.159406,0.290472,0.274704,0.787343,0.101176,0.531017
2,2,0.342358,-0.044282,0.339240,0.330811,0.796982,0.127206,0.541704
3,3,0.359119,-0.026443,0.358427,0.351231,0.832356,0.223314,0.577435
4,4,0.215446,0.018165,0.208882,0.198422,0.784480,0.092379,0.528429
...,...,...,...,...,...,...,...,...
1065,1065,0.193410,0.053807,0.192517,0.183444,0.844898,0.257581,0.575340
1066,1066,0.099270,0.013470,0.098644,0.089926,0.832053,0.222068,0.568146
1067,1067,0.176846,-0.130924,0.170863,0.160646,0.798710,0.132414,0.548549
1068,1068,0.350094,0.027682,0.343700,0.338141,0.779024,0.078339,0.518761


In [182]:
df.max()

example                         1069.000000
similarity normalized              0.532130
similarity raw                     0.378550
sim_norm w/ median threshold       0.529216
sim_norm w/ mean threshold         0.521735
sim w/ ranks                       0.873251
kendalltau                         0.344156
rbo                                0.636516
dtype: float64

In [185]:
df.min()

example                         0.000000
similarity normalized           0.021997
similarity raw                 -0.337544
sim_norm w/ median threshold    0.021838
sim_norm w/ mean threshold      0.018403
sim w/ ranks                    0.731883
kendalltau                     -0.048717
rbo                             0.483627
dtype: float64

In [183]:
df["similarity normalized"].idxmax()

589

In [184]:
df["similarity normalized"].idxmin()

903

In [152]:
df.iloc[589]

example                         589.000000
similarity normalized             0.532130
similarity raw                    0.013085
sim_norm w/ median threshold      0.529216
sim_norm w/ mean threshold        0.521735
sim w/ ranks                      0.792425
kendalltau                        0.114412
rbo                               0.533278
Name: 589, dtype: float64

In [192]:
df2.max()

example                         1069.000000
similarity normalized              0.287572
similarity raw                     0.198161
sim_norm w/ median threshold       0.269140
sim_norm w/ mean threshold         0.250140
sim w/ ranks                       0.845095
kendalltau                         0.266276
rbo                                0.606015
dtype: float64

In [191]:
df.mean()

example                         534.500000
similarity normalized             0.227269
similarity raw                    0.032436
sim_norm w/ median threshold      0.223305
sim_norm w/ mean threshold        0.209596
sim w/ ranks                      0.811874
kendalltau                        0.168294
rbo                               0.557020
dtype: float64

In [190]:
df2.mean()

example                         534.500000
similarity normalized             0.120074
similarity raw                    0.016374
sim_norm w/ median threshold      0.113154
sim_norm w/ mean threshold        0.102037
sim w/ ranks                      0.799616
kendalltau                        0.134198
rbo                               0.542645
dtype: float64

The following block of code iterates through the entire dataset, grabs the appropriate attributions and attentions from their respective dictionaries, and computes the cosine similarities, kendall tau coefficients, and the RBO. The difference is that this block of code masks the values where the input is a non-alphanumeric token (e.g. ".,][?") before calculating similarities.

In [212]:
def get_sim_dataframe_alpha(cogs402_ds, all_attentions, all_attributions):

  dataframe = []

  for i in tqdm(range(len(cogs402_ds))):
    exam_attrib = all_attributions[str(i)]
    attention_final_layer = all_attentions[str(i)]
    
    exam_attrib = exam_attrib[:len(all_attentions[str(i)])]

    #input_ids
    text = cogs402_ds['text'][i]
    input_ids, _, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
    indices = input_ids[0].detach().tolist()
    all_tokens_curr = tokenizer.convert_ids_to_tokens(indices)

    exam_tokens = all_tokens_curr
    alpha_neumeric_nums = [idx for idx, element in enumerate(exam_tokens) if element.isalnum()]
    mask = np.ones(attention_final_layer.shape,dtype=bool) 
    mask[alpha_neumeric_nums] = False

    attention_final_layer[mask] = 0
    exam_attrib[mask] = 0

    #raw attributions
    cosine_raw = np.dot(exam_attrib, attention_final_layer) / (norm(exam_attrib)*norm(attention_final_layer))

    #normalized attributions
    attention_final_layer2 = normalize(attention_final_layer)

    exam_attrib2 = np.abs(exam_attrib)
    exam_attrib2 = normalize(exam_attrib2)
    
    cosine = np.dot(exam_attrib2, attention_final_layer2) / (norm(exam_attrib2)*norm(attention_final_layer2))

    #sim for attributions and attentions above the median
    exam_attrib3 = np.abs(exam_attrib)
    exam_attrib3 = normalize(exam_attrib3)
    median_exam = np.percentile(exam_attrib3, 50)
    exam_attrib3[exam_attrib3 < median_exam] = 0

    attention_final_layer3 = np.copy(attention_final_layer)
    attention_final_layer3 = normalize(attention_final_layer3)
    median_12 = np.percentile(attention_final_layer3, 50)
    attention_final_layer3[attention_final_layer3 < median_12] = 0

    cosine_med = np.dot(exam_attrib3, attention_final_layer3) / (norm(exam_attrib3)*norm(attention_final_layer3))

    #sim for attributions and attentions above the mean
    exam_attrib4 = np.abs(exam_attrib)
    exam_attrib4 = normalize(exam_attrib4)
    mean_exam = np.mean(exam_attrib4)
    exam_attrib4[exam_attrib4 < mean_exam] = 0

    attention_final_layer4 = np.copy(attention_final_layer)
    attention_final_layer4 = normalize(attention_final_layer4)
    mean_12 = np.mean(attention_final_layer4)
    attention_final_layer4[attention_final_layer4 < mean_12] = 0
    
    cosine_mean = np.dot(exam_attrib4, attention_final_layer4) / (norm(exam_attrib4)*norm(attention_final_layer4))

    #sim using the ranks of the tokens
    exam_attrib_rank = np.abs(exam_attrib)
    order_attrib = exam_attrib_rank.argsort()
    ranks_attrib = order_attrib.argsort()

    attention_final_layer_rank = np.copy(attention_final_layer)
    order = attention_final_layer_rank.argsort()
    ranks = order.argsort()

    
    cosine_rank = np.dot(ranks_attrib, ranks) / (norm(ranks_attrib)*norm(ranks))

    tau, p_value = stats.kendalltau(ranks_attrib, ranks)

    rbo_sim = rbo.RankingSimilarity(order_attrib, order).rbo()

    d = {'example': i, 'similarity normalized': cosine, 'similarity raw': cosine_raw, 'sim_norm w/ median threshold':cosine_med, 'sim_norm w/ mean threshold':cosine_mean, "sim w/ ranks":cosine_rank,
        "kendalltau": tau, "rbo":rbo_sim}
    dataframe.append(d)

  return pd.DataFrame(dataframe)

In [214]:
df_alpha = get_sim_dataframe_alpha(cogs402_ds, all_attentions_final, all_attributions)

100%|██████████| 1070/1070 [03:31<00:00,  5.06it/s]


In [216]:
df_alpha2 = get_sim_dataframe_alpha(cogs402_ds, all_attentions_all, all_attributions)

100%|██████████| 1070/1070 [03:30<00:00,  5.08it/s]


In [215]:
df_alpha.mean()

example                         534.500000
similarity normalized             0.400218
similarity raw                    0.045698
sim_norm w/ median threshold      0.382465
sim_norm w/ mean threshold        0.353881
sim w/ ranks                      0.884370
kendalltau                        0.400200
rbo                               0.792063
dtype: float64

In [217]:
df_alpha2.mean()

example                         534.500000
similarity normalized             0.302328
similarity raw                   -0.003185
sim_norm w/ median threshold      0.280655
sim_norm w/ mean threshold        0.257808
sim w/ ranks                      0.876614
kendalltau                        0.374737
rbo                               0.784229
dtype: float64

If you wish to find the unscaled version of the attentions, the following block of code gives two dictionaries of unscaled attentions.

In [218]:
all_attentions_final2 = torch.load('/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/full_attention_matrices/example_atten_dict_12_unscale.pt')
all_attentions_all2 = torch.load('/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/full_attention_matrices/example_atten_dict_all_unscale.pt')

In [ ]:
# all_attentions_final = {}
# all_attentions_all = {}
# for i in tqdm(range(len(cogs402_ds))):
#   if str(i) not in all_attentions_final and str(i) not in all_attentions_all:

#     att_mat = all_attentions[str(i)]

#     # Sum the attentions for the last layer and over all layers
#     attention_final_layer = att_mat[11].sum(axis=0)
#     all_attentions_final[str(i)] = attention_final_layer

#     attention_all_layer = att_mat.sum(axis=1)
#     attention_all_layer = attention_all_layer.sum(axis=0)
#     all_attentions_all[str(i)] = attention_all_layer
    
#     torch.save(all_attentions_final, '/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/full_attention_matrices/example_atten_dict_12_unscale.pt')
#     torch.save(all_attentions_all, '/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/full_attention_matrices/example_atten_dict_all_unscale.pt')

100%|██████████| 1070/1070 [03:07<00:00,  5.72it/s]


In [ ]:
# dataframe_final_layer = []
# dataframe_all_layer = [] 

In [219]:
df_unscale = get_sim_dataframe(cogs402_ds, all_attentions_final2, all_attributions)
df_unscale2 = get_sim_dataframe(cogs402_ds, all_attentions_all2, all_attributions)

100%|██████████| 1070/1070 [00:07<00:00, 144.90it/s]


In [222]:
df_unscale.mean()

example                         534.500000
similarity normalized             0.105481
similarity raw                    0.020551
sim_norm w/ median threshold      0.095462
sim_norm w/ mean threshold        0.076707
sim w/ ranks                      0.777507
kendalltau                        0.077260
rbo                               0.521135
dtype: float64

In [223]:
df_unscale2.mean()

example                         534.500000
similarity normalized             0.059124
similarity raw                    0.000046
sim_norm w/ median threshold      0.053317
sim_norm w/ mean threshold        0.037848
sim w/ ranks                      0.776544
kendalltau                        0.074032
rbo                               0.525490
dtype: float64

In [224]:
df_unscale_alpha = get_sim_dataframe_alpha(cogs402_ds, all_attentions_final2, all_attributions)

100%|██████████| 1070/1070 [03:33<00:00,  5.00it/s]


In [225]:
df_unscale_alpha2 = get_sim_dataframe_alpha(cogs402_ds, all_attentions_all2, all_attributions)

100%|██████████| 1070/1070 [03:34<00:00,  4.99it/s]


In [226]:
df_unscale_alpha.mean()

example                         534.500000
similarity normalized             0.408307
similarity raw                    0.024952
sim_norm w/ median threshold      0.384152
sim_norm w/ mean threshold        0.354161
sim w/ ranks                      0.881266
kendalltau                        0.390058
rbo                               0.789077
dtype: float64

In [227]:
df_unscale_alpha2.mean()

example                         534.500000
similarity normalized             0.235386
similarity raw                   -0.002723
sim_norm w/ median threshold      0.211407
sim_norm w/ mean threshold        0.194379
sim w/ ranks                      0.876572
kendalltau                        0.374509
rbo                               0.783674
dtype: float64